In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier,VotingClassifier,BaggingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.utils import class_weight
from imblearn.over_sampling import RandomOverSampler, SMOTE
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler,StandardScaler,RobustScaler
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier,Pool


pd.pandas.set_option('display.max_columns', None)
import os
import datetime
import gc
import glob
import itertools

## What actually want to do?

## Before scaling the whole data we have to split the data into train and test

In [2]:
df = pd.read_csv('train.csv')
df.head()

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.23868,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,B,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,19.4205,155.868030,14.754720,51.216883,257.432377,30.284345,0.484710,50.628208,6.085041,0.031442,1.113875,1.117800,28.310953,1.357182,37.476568,70.79836,178.553100,0.23868,0.363489,110.581815,75.74548,37.532000,0.005518,1.74307,17.222328,4.926396,0.858603,1111.287150,0.003042,A,109.125159,95.415086,52.260480,17.175984,0.296850,6785.003474,10.358927,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,26.4825,128.988531,219.320160,482.141594,257.432377,32.563713,0.495852,85.955376,5.376488,0.036218,1.050225,0.700350,39.364743,1.009611,21.459644,70.81970,321.426625,0.23868,0.210441,120.056438,65.46984,28.053464,1.289739,1.74307,36.861352,7.813674,8.146651,1494.076488,0.377208,B,109.125159,78.526968,5.390628,224.207424,8.745201,8338.906181,11.626917,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,23.6577,237.282264,11.050410,661.518640,257.432377,15.201914,0.717882,88.159360,2.347652,0.029054,1.400300,0.636075,41.116960,0.722727,21.530392,47.27586,196.607985,0.23868,0.292431,139.824570,71.57120,24.354856,2.655345,1.74307,52.003884,7.386060,3.813326,15691.552180,0.614484,B,31.674357,78.526968,31.323372,59.301984,7.884336,10965.766040,14.852022,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,24.0108,324.546318,149.717165,6074.859475,257.432377,82.213495,0.536467,72.644264,30.537722,0.025472,1.050225,0.693150,31.724726,0.827550,34.415360,74.06532,200.178160,0.23868,0.207708,97.920120,52.83888,26.019912,1.144902,1.74307,9.064856,7.350720,3.490846,1403.656300,0.164268,B,109.125159,91.994825,51.141336,29.102640,4.274640,16198.049590,13.666727,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


In [3]:
df.shape

(617, 58)

In [4]:
df.drop('Id',axis = 1, inplace = True)

In [5]:
df['EJ'] = df['EJ'].map({'B': 0, 'A': 1})

## Splitting the data

In [6]:
X = df.iloc[:,:-1]
y = df.iloc[:,-1]

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [8]:
X_train['Class'] = y_train

In [9]:
X_train

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
408,0.252107,6382.61448,410.629350,6.073217,17.952834,4.996476,0.038671,14.689962,9.572136,7451.76164,21.8922,308.162583,82.166910,823.166067,1145.909251,12.499760,0.956602,76.647904,5.831118,0.027064,5.461170,0.884175,43.526477,1.020645,42.579716,71.96236,431.334795,2.976129,0.330693,161.465790,87.83796,20.744960,2.531199,7.00595,14.324988,9.502926,4.212395,1813.654538,0.644904,0,55.844646,96.165875,71.723964,25.109952,9.718869,6675.741561,10.645603,7.020590,2.00158,0.629889,26.706042,72.611063,8776.686879,19.125940,145.114812,0.171564,1
433,0.679407,5885.36377,85.200147,9.427268,8.138688,2.294481,0.025578,8.079164,1.229900,3196.66317,27.8949,235.417104,39.041135,804.108692,709.196735,23.198084,0.757570,36.532224,6.516863,0.031442,1.050225,0.175275,18.848979,0.452394,7.030640,12.94562,109.633200,0.238680,0.142116,60.232470,17.16564,13.347744,31.402041,1.74307,14.572048,9.305022,0.826355,185.594100,3.218436,0,109.125159,97.797219,3.828384,18.672528,74.699334,6078.639486,6.014683,38.083826,1.35662,0.067730,15.766890,118.826134,384.850764,16.435657,14.158860,0.069015,1
33,0.162374,3245.45474,85.200147,14.469184,8.138688,4.208025,0.038671,8.009842,1.229900,4553.78065,28.2480,126.211986,102.845400,51.216883,257.432377,76.524266,0.653307,90.569472,9.063700,0.019502,1.050225,0.887175,31.498071,1.599930,7.030640,25.96884,569.130245,1.798173,0.319761,136.080277,116.36044,32.768848,0.006897,1.74307,0.988240,21.790644,1.842167,1488.997413,0.073008,0,109.125159,96.392966,6501.264480,25.109952,18.523440,15103.820970,9.350048,6.169800,0.88885,0.331877,17.915652,1228.992706,4256.400636,15.803087,105.475148,7.128000,0
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,22.5984,175.638726,152.707705,823.928241,257.432377,47.223358,0.563481,23.387600,4.851915,0.023482,1.050225,0.069225,13.784111,1.302012,36.205956,69.08340,295.570575,0.238680,0.284232,89.245560,84.31664,29.657104,5.310690,1.74307,23.187704,7.294176,1.987283,1433.166750,0.949104,0,30.879420,78.526968,3.828384,13.394640,10.265073,9028.291921,3.583450,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
610,0.581128,4268.30888,85.200147,43.212441,11.835708,6.343044,0.025578,7.045636,11.188576,5719.34587,27.8949,246.205359,99.000420,398.701688,257.432377,72.434271,0.488119,23.387600,34.323767,0.050546,1.050225,0.069225,39.389152,1.456488,39.390536,44.90324,502.246010,1.061424,0.554799,219.902355,118.23452,26.381584,0.005518,1.74307,0.804068,16.475508,0.286201,2376.085275,0.003042,1,44.679063,78.526968,53.202384,113.400336,0.296850,14578.902650,3.583450,0.173229,1.20640,0.196417,17.255210,72.611063,38475.318010,27.766102,133.988340,21.978000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,0.602493,5557.12344,100.797966,46.904065,8.138688,5.767209,0.025578,12.389732,9.347240,7900.15245,18.7143,136.741662,NaN,974.971771,257.432377,12.646816,0.629668,83.759320,18.995607,0.027064,1.304825,0.673875,38.946303,1.837161,7.030640,48.38748,379.117105,0.632502,0.519270,158.856645,94.98448,13.668472,0.524172,1.74307,47.610708,6.149160,5.155649,1212.940950,0.048672,0,NaN,81.307668,8.391048,42.354480,4.316199,5136.721022,9.625698,18.799615,0.49706,0.237055,26.733948,103.216456,642.540600,32.149440,39.436476,0.062526,0
106,0.158101,5543.36090,85.200147,12.409679,10.241532,3.694203,0.025578,11.885572,6.444676,5860.19359,12.0054,198.486936,69.087205,292.469150,257.432377,64.052079,1.142518,64.486352,8.438775,0.019502,1.050225,0.915450,32.244289,1.848195,37.746680,56.33372,484.359655,0.238680,0.374421,167.919000,138.09044,22.839928,0.310365,1.74307,13.934184,6.7

In [10]:
y_train

408    1
433    1
33     0
0      1
610    0
      ..
71     0
106    0
270    0
435    0
102    0
Name: Class, Length: 431, dtype: int64

## Undersampling the data for solving class imbalance

In [11]:
# Separate majority (class 0) and minority (class 1) samples
majority_class = 0
minority_class = 1

df_majority = X_train[X_train['Class'] == majority_class]
df_minority = X_train[X_train['Class'] == minority_class]

# Determine the number of samples to be undersampled (match minority class count)
num_minority_samples = df_minority.shape[0]

# Sample randomly from majority class to match the number of minority samples
df_majority_undersampled = df_majority.sample(n=num_minority_samples, random_state=42)

# Concatenate the undersampled majority class with the minority class to create the balanced DataFrame
df_balanced = pd.concat([df_majority_undersampled, df_minority], axis=0)

In [12]:
df_balanced

,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,BN,BP,BQ,BR,BZ,CB,CC,CD,CF,CH,CL,CR,CS,CU,CW,DA,DE,DF,DH,DI,DL,DN,DU,DV,DY,EB,EE,EG,EH,EJ,EL,EP,EU,FC,FD,FE,FI,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
586,0.807597,1750.88452,85.200147,75.185869,8.138688,5.386272,0.025578,11.639794,2.519538,3957.98233,22.9515,369.344070,26.680410,1021.984599,257.432377,32.094972,0.658913,101.462544,2.548358,0.040994,1.050225,0.587475,38.724879,1.826127,38.523804,71.22904,270.876495,2.386449,0.308829,145.603845,78.37640,24.757472,10.317912,1.74307,29.876292,13.111140,0.306356,2158.103788,1.508832,0,30.247503,78.526968,30.700500,51.145248,28.907253,8801.391261,9.950965,18.019869,1.02341,0.209963,13.236746,72.611063,5252.358249,30.750344,38.266216,0.098471,0
447,0.871692,1212.81567,310.545522,119.150881,8.138688,6.484788,0.174479,10.165126,6.782020,7436.25305,13.4178,292.609692,13.939355,1489.886847,1525.211415,67.599805,0.434789,198.057296,10.030739,0.054924,1.521235,0.926850,55.221876,0.675832,7.030640,31.21266,261.806920,0.458406,0.221373,128.151187,69.96636,19.564408,0.358644,1.74307,46.761720,11.563248,5.075029,1390.166325,0.231192,0,30.166422,205.275909,30.806844,41.651904,4.411191,4903.479720,10.601499,3.345836,1.38040,0.758576,33.626730,72.611063,5251.885857,17.964988,12.214428,0.434077,0
144,0.346113,1819.38504,145.666734,19.498712,8.138688,4.899027,0.027710,10.234448,2.765518,5845.53993,17.6550,135.160515,42.341670,753.953409,1695.643929,49.263760,0.644877,77.076016,12.124467,0.020696,1.572155,0.742725,38.088501,0.959958,36.925212,87.42998,227.174005,11.539827,0.420882,97.773285,87.43728,29.083888,0.082764,1.74307,12.600060,6.477822,6.030376,1309.627138,0.085176,0,27.999114,112.873248,117.117660,32.295648,1.531746,4378.482797,17.421080,1.514886,1.32037,0.196417,18.110994,85.181845,12851.988610,31.475939,17.672212,0.693000,0
241,0.213650,2923.50278,85.200147,12.394194,8.138688,4.482654,0.025578,4.613064,1.229900,4280.19506,23.3046,174.528108,47.697550,520.891488,257.432377,12.499760,0.675042,67.609984,0.510888,0.028258,1.050225,0.613800,42.639036,1.467522,64.521624,51.71652,256.010375,0.238680,0.347091,102.317640,124.63904,20.799552,0.889713,1.74307,12.883056,11.732880,2.128368,1698.815025,0.164268,0,38.497602,78.526968,69.012192,35.151648,3.152547,6526.990998,10.154946,5.300579,1.20292,0.067730,18.287732,72.611063,2084.744628,29.421947,49.562440,0.124326,0
19,0.132463,2160.54454,85.200147,20.300835,8.138688,4.810437,0.025578,9.371074,5.158552,4589.92741,9.8868,117.950175,37.879305,51.216883,257.432377,39.778648,0.389061,60.855328,6.389141,0.015920,1.050225,0.674775,31.208650,1.136502,7.030640,54.53728,334.119595,0.238680,0.404484,122.637345,62.48064,29.172600,0.005518,1.74307,11.382728,4.926396,1.479377,1237.071075,0.003042,1,109.125159,78.526968,3.828384,16.005024,0.296850,27186.486980,14.818944,0.173229,1.00514,0.419926,11.618198,72.611063,21451.810610,37.265815,51.308828,21.978000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189,0.397389,3894.48906,85.200147,36.860494,10.760688,7.352970,0.025578,11.520056,8.289526,5292.52007,24.0108,150.247116,32.929805,501.997436,634.167940,30.440592,0.672233,88.278280,18.396529,0.023880,1.152065,0.683475,35.544734,1.450971,21.058616,35.37590,278.349470,0.238680,0.412683,269.937322,78.20892,25.562704,17.801157,1.74307,13.368192,8.273094,2.575809,1449.114925,3.200184,0,109.125159,78.526968,24.274284,22.796592,23.195859,72329.133120,13.159531,8.954666,0.95004,0.656981,21.934116,211.072970,1896.743547,26.393053,139.826780,0.121055,1
313,0.820416,3457.44795,173.657460,17.048985,8.138688,6.148146,0.025578,13.070348,1.229900,5551.52391,24.3639,223.971804,136.525445,344.789976,257.432377,41.791477,0.529998,76.893672,2.565084,0.024676,1.425760,1.008225,33.612937,1.202706,7.030640,70.82940,285.977670,0.238680,0.467343,128.591692,113.17620,24.818888,

In [13]:
X_train = df_balanced.iloc[:,:-1]
y_train = df_balanced.iloc[:,-1]

## Notice we don't touch the splitting test data and it is as it is

In [14]:
scalar = MinMaxScaler()
XS_train = scalar.fit_transform(X_train.values)
XS_test = scalar.transform(X_test.values)

In [15]:
model = XGBClassifier()
model.fit(XS_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [16]:
y_pred = model.predict(XS_test)

In [17]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

In [18]:
print(f'xgb-{accuracy}-{precision}-{recall}-{f1}')

xgb-0.8817204301075269-0.6428571428571429-0.9473684210526315-0.7659574468085106
